## Metodo con Scrapy

### Nota
El metodo de scrappy solo puede correr una sola vez, en caso de correrlo una segunda vez es necesario abortar el runtime y correrlo nuevamente

In [ ]:
# Descargamos la librería Scrapy en caso de que no la tengamos
!pip install scrapy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.0 MB/s eta 0:00:00


In [ ]:
from twisted.internet import reactor  # Importamos el reactor de Twisted, necesario para controlar el ciclo de eventos de Scrapy
from scrapy.crawler import CrawlerRunner  # Importamos CrawlerRunner para gestionar el ciclo de vida del spider en Scrapy
from scrapy.utils.log import configure_logging  # Importamos configure_logging para configurar el sistema de logging de Scrapy
import scrapy  # Importamos la biblioteca Scrapy que usaremos para crear el spider
import pprint  # Importamos pprint para imprimir los datos de forma más legible

class MercadolibreSpider(scrapy.Spider):
    name = 'mercadolibre_spider'
    allowed_domains = ['mercadolibre.com.co']
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    start_urls = ['https://listado.mercadolibre.com.co/portatil']
    cantidad_productos = 10  # Definimos la cantidad de productos a obtener

    def parse(self, response):
        # Seleccionamos los enlaces de los primeros 'cantidad_productos' resultados de la búsqueda
        productos = response.css('li.ui-search-layout__item a::attr(href)').getall()[:self.cantidad_productos]

        # Iteramos sobre cada URL de producto y hacemos una solicitud a cada uno
        for producto_url in productos:
            yield response.follow(producto_url, self.parse_product)

    def parse_product(self, response):
        data = {}

        try:
            data['Categoria'] = response.css('div.ui-pdp-breadcrumb::text').get()
        except:
            data['Categoria'] = None

        try:
            data['Titulo'] = response.css('h1.ui-pdp-title::text').get()
        except:
            data['Titulo'] = None

        try:
            data['Precio'] = response.css('div.ui-pdp-price__second-line span.andes-money-amount__fraction::text').get()
        except:
            data['Precio'] = None

        try:
            data['Descuento'] = bool(response.css('s[role="img"][aria-label^="Antes:"]'))
        except:
            data['Descuento'] = None

        try:
            vendedor_data = response.css('button.ui-pdp-seller__link-trigger-button span::text').getall()
            data['Vendedor'] = vendedor_data[1] if len(vendedor_data) > 1 else None
        except:
            data['Vendedor'] = None

        try:
            data['Calificacion promedio'] = response.css('span.ui-pdp-review__rating[aria-hidden="true"]::text').get()
        except:
            data['Calificacion promedio'] = None

        try:
            data['Cantidad de Calificaciones'] = response.css('span.ui-pdp-review__amount[aria-hidden="true"]::text').get()
        except:
            data['Cantidad de Calificaciones'] = None

        try:
            data['Garantia'] = response.xpath("//p[contains(., 'garantía de fábrica.')]/text()").get()
        except:
            data['Garantia'] = None

        try:
            data['Descripcion'] = response.css('p.ui-pdp-description__content::text').get()
        except:
            data['Descripcion'] = None

        try:
            data['Stock'] = response.css('p.ui-pdp-stock-information__title::text').get()
        except:
            data['Stock'] = None

        try:
            data['Cantidad de Opiniones'] = response.css('span.total-opinion::text').get()
        except:
            data['Cantidad de Opiniones'] = None

        try:
            data['Numero de Publicacion'] = response.css('span.ui-pdp-color--BLACK.ui-pdp-family--SEMIBOLD::text').get()
        except:
            data['Numero de Publicacion'] = None

        data['URL del Producto'] = response.url

        # Imprimimos los datos del producto
        pprint.pprint(data)


# Configuración de logging
configure_logging()

# Creación de instancia de CrawlerRunner
runner = CrawlerRunner()

# Verificamos si el reactor ya está en ejecución antes de iniciarlo
if not reactor.running:
    # Ejecutamos el spider MercadolibreSpider
    d = runner.crawl(MercadolibreSpider)
    # Detenemos el reactor cuando el spider termina
    d.addBoth(lambda _: reactor.stop())
    # Iniciamos el ciclo de eventos del reactor
    reactor.run()


INFO:scrapy.addons:Enabled addons:
[]
2024-11-01 01:13:24 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
INFO:scrapy.extensions.telnet:Telnet Password: abbde5c4b69c3c7e
2024-11-01 01:13:24 [scrapy.extensions.telnet] INFO: Telnet Password: abbde5c4b69c3c7e
INFO:scrapy.middleware:Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage

{'Calificacion promedio': '4.8',
 'Cantidad de Calificaciones': '(55)',
 'Cantidad de Opiniones': '16 comentarios',
 'Categoria': None,
 'Descripcion': 'Aviso legal',
 'Descuento': True,
 'Garantia': '12 meses de garantía de fábrica.',
 'Numero de Publicacion': '24 GB',
 'Precio': '1.779.309',
 'Stock': 'Stock disponible',
 'Titulo': 'Potátil V-Series V14 14" color gris 24GB de Ram - 512GB SSD - '
           'Intel Core i5',
 'URL del Producto': 'https://www.mercadolibre.com.co/potatil-v-series-v14-14-color-gris-24gb-de-ram-512gb-ssd-intel-core-i5/p/MCO36489354',
 'Vendedor': 'DISTRIMAK '}
{'Calificacion promedio': '4.7',
 'Cantidad de Calificaciones': '(442)',
 'Cantidad de Opiniones': '127 comentarios',
 'Categoria': None,
 'Descripcion': 'Aviso legal',
 'Descuento': True,
 'Garantia': '12 meses de garantía de fábrica.',
 'Numero de Publicacion': '16 GB',
 'Precio': '1.739.900',
 'Stock': 'Stock disponible',
 'Titulo': 'Portatil Asus E1504fa-nj474 Ryzen 5-7520u Ram 16gb Ssd 512gb Col

DEBUG:scrapy.core.engine:Crawled (200) <GET https://articulo.mercadolibre.com.co/MCO-1133597253-laptop-hp-240-g8-intel-celeron-n4120-8gb-256gb-ssd-windows10-_JM#is_advertising=true&position=1&search_layout=stack&type=pad&tracking_id=8e00f9c3-a277-4dd0-a7fb-f62c6db47db4&is_advertising=true&ad_domain=VQCATCORE_LST&ad_position=1&ad_click_id=NWYzNGNiMzQtYWE4NS00Nzc1LWI1NWQtMThiZjZkNGM2MjYx> (referer: https://listado.mercadolibre.com.co/portatil)
2024-11-01 01:13:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://articulo.mercadolibre.com.co/MCO-1133597253-laptop-hp-240-g8-intel-celeron-n4120-8gb-256gb-ssd-windows10-_JM#is_advertising=true&position=1&search_layout=stack&type=pad&tracking_id=8e00f9c3-a277-4dd0-a7fb-f62c6db47db4&is_advertising=true&ad_domain=VQCATCORE_LST&ad_position=1&ad_click_id=NWYzNGNiMzQtYWE4NS00Nzc1LWI1NWQtMThiZjZkNGM2MjYx> (referer: https://listado.mercadolibre.com.co/portatil)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://articulo.mercadolibre.com.co/MCO-

{'Calificacion promedio': '4.7',
 'Cantidad de Calificaciones': '(302)',
 'Cantidad de Opiniones': '104 comentarios',
 'Categoria': None,
 'Descripcion': 'Convierte las tareas cotidianas en algo especial con Vivobook '
                '15, tu herramienta esencial para hacer las cosas más '
                'fácilmente, en cualquier lugar. También es completamente '
                'fácil de usar, con su bisagra plana de 180° y el protector '
                'físico de la cámara web. ASUS Antimicrobial Guard Plus '
                'protege las superficies que se tocan con frecuencia de '
                'bacterias dañinas, salvaguardando su salud. ¡Haz que todos '
                'tus días sean más agradables con Vivobook 15!',
 'Descuento': True,
 'Garantia': '12 meses de garantía de fábrica.',
 'Numero de Publicacion': '20 GB',
 'Precio': '2.021.137',
 'Stock': 'Stock disponible',
 'Titulo': 'Portatil Asus Vivobook X1504za-nj372 Corei5 1235u 20gb 512gb Azul ',
 'URL del Producto': 'htt

INFO:scrapy.core.engine:Closing spider (finished)
2024-11-01 01:13:29 [scrapy.core.engine] INFO: Closing spider (finished)
INFO:scrapy.statscollectors:Dumping Scrapy stats:
{'downloader/request_bytes': 9009,
 'downloader/request_count': 13,
 'downloader/request_method_count/GET': 13,
 'downloader/response_bytes': 1152680,
 'downloader/response_count': 13,
 'downloader/response_status_count/200': 10,
 'downloader/response_status_count/301': 1,
 'downloader/response_status_count/302': 2,
 'dupefilter/filtered': 1,
 'elapsed_time_seconds': 3.947109,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2024, 11, 1, 1, 13, 29, 558319, tzinfo=datetime.timezone.utc),
 'httpcompression/response_bytes': 6564543,
 'httpcompression/response_count': 10,
 'log_count/DEBUG': 16,
 'log_count/INFO': 10,
 'memusage/max': 194367488,
 'memusage/startup': 194367488,
 'request_depth_max': 1,
 'response_received_count': 10,
 'scheduler/dequeued': 13,
 'scheduler/dequeued/memory': 13,
 'scheduler/

{'Calificacion promedio': '4.6',
 'Cantidad de Calificaciones': '(383)',
 'Cantidad de Opiniones': '132 comentarios',
 'Categoria': None,
 'Descripcion': 'Sistema operativo: FreeDos (WINDOWS 10 PRE-INSTALADO)',
 'Descuento': True,
 'Garantia': '12 meses de garantía de fábrica.',
 'Numero de Publicacion': 'NEW TECNOLOGIES',
 'Precio': '897.900',
 'Stock': None,
 'Titulo': 'Laptop Hp 240 G8 Intel Celeron N4120 8gb 256gb Ssd Windows10',
 'URL del Producto': 'https://articulo.mercadolibre.com.co/MCO-1133597253-laptop-hp-240-g8-intel-celeron-n4120-8gb-256gb-ssd-windows10-_JM',
 'Vendedor': 'NEW TECNOLOGIES'}
{'Calificacion promedio': '4.0',
 'Cantidad de Calificaciones': '(1)',
 'Cantidad de Opiniones': '1 comentario',
 'Categoria': None,
 'Descripcion': 'PORTÁTIL',
 'Descuento': True,
 'Garantia': None,
 'Numero de Publicacion': '+100',
 'Precio': '524.999',
 'Stock': 'Stock disponible',
 'Titulo': 'Portatil Hp Probook 450 G2 Core I5 4ta 8gb Ssd 240gb '
           '(Reacondicionado)',
 'UR

##Metodología empleada de Scraping

Si bien en el ejercicio realizamos pruebas con las técnicas indicadas, lo que nos permitió trabajar con cada una de ellas y validar su comportamiento y utilidad, elegimos Selenium, ya que es ideal para interactuar con páginas dinámicas donde los datos se cargan dinámicamente. Aunque consume más recursos para su procesamiento, Selenium es útil para realizar pruebas o automatizar la navegación y las acciones de clics. Es flexible para usarse en múltiples navegadores y simula la interacción del usuario.


##Resultados y Conclusiones

Cada una de estas herramientas ofrece ventajas únicas que nos permiten, de diferentes maneras, obtener datos valiosos para el análisis en páginas web. Por ejemplo, BeautifulSoup es ideal para el análisis de contenido HTML, facilitando la extracción de información estructurada de documentos web. Por otro lado, Selenium permite la automatización de navegadores, lo que es útil para interactuar con páginas que requieren acciones dinámicas, como clics y desplazamientos. Scrapy es un framework que facilita la construcción de spiders para la recolección masiva de datos, optimizando el proceso de scraping.



A través de estas técnicas, se pudieron analizar datos dinámicos como precios, disponibilidad y características de los computadores, interactuar con los elementos de la página web, y validar la eficiencia del sitio y sus restricciones. Al mismo tiempo, se identificaron posibles mejoras en el sitio, actuando siempre desde la ética y las políticas de legalidad del mismo.




### Resultados y conclusiones
Dar una breve explicacion sobre el archivo robots.txt de Mercadolibre:
https://www.mercadolibre.com.co/robots.txt
